In [1]:
!pip install xdggs
!pip install --upgrade intake==0.7 intake_xarray==0.7 

In [1]:
from distributed import LocalCluster

cluster = LocalCluster()
#cluster = LocalCluster(n_workers=2, memory_limit="5GiB",threads_per_worker=1)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41615,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:34695,Total threads: 2
Dashboard: http://127.0.0.1:38093/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:38945,


In [3]:
import intake
import numpy as np
import xdggs

#For level ≤ 14, 12 × 4^level fits in uint32.
#For level ≥ 15,  not fit anymore, use int64 or uint64.

level=15 # crash due to memory on pangeo-eosc
level=12
#chunk_size=12 * 4 ** (level-2)
chunk_size=12 * 4 ** (11)
cell_ids = np.arange(12 * 4 ** level, dtype=np.int32)

cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")["online"]
ds = cat.nicam_220m_test(zoom=level, chunks={"cell": chunk_size}).to_dask()
ds =ds.assign(windspeed=np.hypot(ds.ss_u10m, ds.ss_v10m))[['windspeed']]
ds = (
    ds.assign_coords(
        cell_ids=(
            "cell",
            cell_ids,
            {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"},
        )
    )
    .pipe(xdggs.decode)
)
ds



/srv/conda/envs/notebook/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 2GB
Dimensions:    (time: 1, cell: 201326592)
Coordinates:
  * cell_ids   (cell) int32 805MB 0 1 2 3 ... 201326589 201326590 201326591
Dimensions without coordinates: time, cell
Data variables:
    windspeed  (time, cell) float32 805MB dask.array<chunksize=(1, 50331648), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=12, indexing_scheme=nested)

In [4]:
#This is where you define the lat/lon limit
lat = np.arange(29, 55, 0.01,) #dtype=np.float32)
lon = np.arange(125.8, 151, 0.01,)# dtype=np.float32)

# Repeat lat and tile lon
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))
ds_regional=ds.dggs.sel_latlon(longitude=full_lon, latitude=full_lat)

ds_regional

<xarray.Dataset> Size: 52MB
Dimensions:    (time: 1, cell: 6554600)
Coordinates:
  * cell_ids   (cell) int32 26MB 20590196 20590175 ... 30409997 30409997
Dimensions without coordinates: time, cell
Data variables:
    windspeed  (time, cell) float32 26MB dask.array<chunksize=(1, 6554600), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=12, indexing_scheme=nested)

In [5]:
ds_regional.windspeed.compute().dggs.explore(alpha=0.8,cmap='Greys_r')

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 25.01 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
